# (i) #

In [10]:
import sympy as sp
# Define a 3x3 world
# Define the grid size
grid_size = 3

# Step 1: Define propositional symbols
P = {}
W = {}
B = {}
S = {}

for x in range(1, grid_size+1):
    for y in range(1, grid_size+1):
        P[(x, y)] = sp.symbols(f'P_{x}_{y}')
        W[(x, y)] = sp.symbols(f'W_{x}_{y}')
        B[(x, y)] = sp.symbols(f'B_{x}_{y}')
        S[(x, y)] = sp.symbols(f'S_{x}_{y}')

P

{(1, 1): P_1_1,
 (1, 2): P_1_2,
 (1, 3): P_1_3,
 (2, 1): P_2_1,
 (2, 2): P_2_2,
 (2, 3): P_2_3,
 (3, 1): P_3_1,
 (3, 2): P_3_2,
 (3, 3): P_3_3}

In [ ]:
# Step 2: Define rules as logical expressions
def get_neighbors(x, y, grid_size):
    """Return valid neighboring cells (up/down/left/right) in an n x n grid."""
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    neighbors = [(x + dx, y + dy) for dx, dy in directions]
    """
    Equivalent to:
    neighbors = []
    for dx, dy in directions:
        neighbors.append((x + dx, y + dy))
    """
    return [(nx, ny) for nx, ny in neighbors if 1 <= nx <= grid_size and 1 <= ny <= grid_size]

def Pit_implies_Breeze(x,y,P,B,grid_size):
    """
    Rule: If there is a pit at (x,y), then agent feels a breeze in all adjacent cells.
    """
    adjacent_Breeze = sp.And(*[B[adj] for adj in get_neighbors(x, y, grid_size)])
    # *(splat operator) unpacks the list into separate arguments.
    # example: sp.And(*[a,b,c]) is equivalent to sp.And(a,b,c)
    pit_implies_breeze = sp.Implies(P[(x,y)], adjacent_Breeze)
    return pit_implies_breeze

def Breeze_for_Pit(x,y,B,P,grid_size):
    """
    Rule: If there is a breeze at (x,y), then there is a pit in at least one of the adjacent cells.
          If there is a pit in at least one of the adjacent cells of (x,y), then there is a breeze at (x,y).
    """
    adjacent_pit = sp.Or(*[P[adj] for adj in get_neighbors(x, y, grid_size)])
    breeze_for_pit = sp.Equivalent(B[(x,y)], adjacent_pit)
    return breeze_for_pit

def Wumpus_implies_Stench(x,y,W,S,grid_size):
    """
    Rule: If there is a wumpus at (x,y), then agent smells a stench in all adjacent cells.
    """
    adjacent_stench = sp.And(*[S[adj] for adj in get_neighbors(x, y, grid_size)])
    wumpus_implies_stench = sp.Implies(W[(x,y)], adjacent_stench)
    return wumpus_implies_stench

def Stench_for_Wumpus(x,y,S,W,grid_size):
    """
    Rule: If there is a stench at (x,y), then there is a wumpus in at least one of the adjacent cells.
          If there is a wumpus in at least one of the adjacent cells of (x,y), then there is a stench at (x,y).
    """
    adjacent_wumpus = sp.Or(*[W[adj] for adj in get_neighbors(x, y, grid_size)])
    stench_for_wumpus = sp.Equivalent(S[(x,y)], adjacent_wumpus)
    return stench_for_wumpus

def at_least_one_pit(P,grid_size):
    return sp.Or(*[P[(x,y)] for x in range(1, grid_size+1) for y in range(1, grid_size+1)])

def at_least_one_wumpus(W,grid_size):
    return sp.Or(*[W[(x,y)] for x in range(1, grid_size+1) for y in range(1, grid_size+1)])


In [ ]:
# Step 3: Combine all rules into a knowledge base
rules = set()

for x in range(1, grid_size+1):
    for y in range(1, grid_size+1):
        pit_implies_breeze = Pit_implies_Breeze(x,y,P,B,grid_size)
        rules.add(pit_implies_breeze)
        breeze_for_pit = Breeze_for_Pit(x,y,B,P,grid_size)
        rules.add(breeze_for_pit)
        wumpus_implies_stench = Wumpus_implies_Stench(x,y,W,S,grid_size)
        rules.add(wumpus_implies_stench)     
        stench_for_wumpus = Stench_for_Wumpus(x,y,S,W,grid_size)
        rules.add(stench_for_wumpus)
rules.add(at_least_one_pit(P, grid_size))
rules.add(at_least_one_wumpus(W, grid_size))
rules

{Equivalent(B_1_1, P_1_2 | P_2_1),
 Equivalent(B_1_2, P_1_1 | P_1_3 | P_2_2),
 Equivalent(B_1_3, P_1_2 | P_2_3),
 Equivalent(B_2_1, P_1_1 | P_2_2 | P_3_1),
 Equivalent(B_2_2, P_1_2 | P_2_1 | P_2_3 | P_3_2),
 Equivalent(B_2_3, P_1_3 | P_2_2 | P_3_3),
 Equivalent(B_3_1, P_2_1 | P_3_2),
 Equivalent(B_3_2, P_2_2 | P_3_1 | P_3_3),
 Equivalent(B_3_3, P_2_3 | P_3_2),
 Equivalent(S_1_1, W_1_2 | W_2_1),
 Equivalent(S_1_2, W_1_1 | W_1_3 | W_2_2),
 Equivalent(S_1_3, W_1_2 | W_2_3),
 Equivalent(S_2_1, W_1_1 | W_2_2 | W_3_1),
 Equivalent(S_2_2, W_1_2 | W_2_1 | W_2_3 | W_3_2),
 Equivalent(S_2_3, W_1_3 | W_2_2 | W_3_3),
 Equivalent(S_3_1, W_2_1 | W_3_2),
 Equivalent(S_3_2, W_2_2 | W_3_1 | W_3_3),
 Equivalent(S_3_3, W_2_3 | W_3_2),
 Implies(P_1_1, B_1_2 & B_2_1),
 Implies(P_1_2, B_1_1 & B_1_3 & B_2_2),
 Implies(P_1_3, B_1_2 & B_2_3),
 Implies(P_2_1, B_1_1 & B_2_2 & B_3_1),
 Implies(P_2_2, B_1_2 & B_2_1 & B_2_3 & B_3_2),
 Implies(P_2_3, B_1_3 & B_2_2 & B_3_3),
 Implies(P_3_1, B_2_1 & B_3_2),
 Implies(P

# (ii) #


In [ ]:
KB = rules.copy()

current_loc = (1,1)
# update knowledge base with percepts and initial conditions.
KB.add(sp.Not(B[current_loc]))
KB.add(sp.Not(S[current_loc]))
KB.add(sp.Not(P[current_loc]))
KB.add(sp.Not(W[current_loc]))

KB_AND=sp.And(*KB)

# ask: is there definitely a wumpus at (2,2), i.e., does KB entail W[2,2]?
query = W[(2,2)]

# check entailment = check if KB AND NOT query is unsatisfiable (contradiction)
entails = not sp.satisfiable(sp.And(KB_AND, sp.Not(query)))
entails

False

In [14]:
# Need to also check if KB entails (not W[2,2]) to be conclusive
entails_not = not sp.satisfiable(sp.And(KB_AND, query))
entails_not

False

In [ ]:
# ask: is there definitely a pit at (2,2), i.e., does KB entail P[2,2]?
query = P[(2,2)]

# define a function to check if query is definitely true, definitely false, or uncertain given a knowledge base.
def Infer(KB_AND, query):
    entails = not sp.satisfiable(sp.And(KB_AND, sp.Not(query)))
    if entails:
       print(query, "is definitely true.")
    else:
        entails_not = not sp.satisfiable(sp.And(KB_AND, query))
        if entails_not:
            print(query, "is definitely false.")
        else:
            print(query,"is uncertain.")

Infer(KB_AND, query)

P_2_2 is uncertain.


# (iii)

In [ ]:
# Infer about neighbors of current location (1,1)
current_loc = (1,1)
neighbors= get_neighbors(current_loc[0],current_loc[1],grid_size)

for neighbor in neighbors:
    Infer(KB_AND, W[neighbor])
    Infer(KB_AND, P[neighbor])


W_2_1 is definitely false.
P_2_1 is definitely false.
W_1_2 is definitely false.
P_1_2 is definitely false.


In [17]:
# update knowledge base with inference results.
KB.add(sp.Not(P[(2,1)]))
KB.add(sp.Not(W[(2,1)]))
KB.add(sp.Not(P[(1,2)]))
KB.add(sp.Not(W[(1,2)]))

In [18]:
# Move to (2,1) and Infer about neighbors of (2,1)
current_loc = (2,1)
# update knowledge base with percepts.
KB.add(B[current_loc])
KB.add(sp.Not(S[current_loc]))

KB_AND=sp.And(*KB)

neighbors= get_neighbors(current_loc[0],current_loc[1],grid_size)
for neighbor in neighbors:
    Infer(KB_AND, W[neighbor])
    Infer(KB_AND, P[neighbor])


W_1_1 is definitely false.
P_1_1 is definitely false.
W_3_1 is definitely false.
P_3_1 is uncertain.
W_2_2 is definitely false.
P_2_2 is uncertain.
